# Introduction

In every aspect of our everyday life we rely on machines to perform the most diverse tasks. From the car that we use for commuting 
to the electric lines that serve our home, from the pacemaker that keeps our heart beating to the laptop we work with. 
Although very different, all these machines inevitably deteriorate over time, and eventually are going to stop functioning. 
The consequences of a failure depend on the specific task and may largely vary in magnitude. 
A car malfunctioning may lead to an accident, an electrical line failing may overcharge the network and cause major blackouts, 
a pacemaker stopping may coerce the patient into an urgent recovery, and a laptop not working may damage hours of work.

In general the “cost” incurred after a failure is tremendously larger than what has to be paid to maintain or even replace the 
specific machine. This is the reason why regular inspections are usually scheduled to track the functioning of the machines, and replacements 
are suggested as they get too old.

An easily-implementable policy consists in **_preventive replacement_**: machines are checked and replaced regularly, 
thus minimizing the cost due to unplanned events. The efficiency of this policy largely depends on how much is known
about the time-to-failure distribution.  
In real-life system the state of the machine evolves continuously but observations are sampled only in a discrete manner (continuous/disrete models), 
at the moment an inspection is scheduled. These models have been largely studied in literature [^J07](https://www.elsevier.com/books/stochastic-processes-and-filtering-theory/jazwinski/978-0-12-381550-7) [^MJ03](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.90.8887&rep=rep1&type=pdf).
It is arguable that when the time-to-failure distribution is sufficiently known, an optimal preventive replacement policy 
is the best offline policy we can aim for, i.e., the best policy that cannot be modified in between two inspections. 

However, we are now experiencing a paradigm shift due to the explosion of Internet of Things [^a](https://www.cisco.com/c/dam/en_us/about/ac79/docs/innov/IoT_IBSG_0411FINAL.pdf) and the large availability of data [^b](https://www.forbes.com/sites/gilpress/2017/11/09/10-predictions-for-ai-big-data-and-analytics-in-2018/#170874aa403c). It is now easy to place sensors in machines so as to capture their metrics almost real-time and we are moving toward quasi-continuous models. In specific cases, the metrics sensed can then be used to establish whether a machine is approaching a failure.
The continuous monitoring of the machine is thus used to raise warnings and eventually schedule additional checking and replacements which can be
carried out in parallel to the regular (preventive replacement based) inspections.
This policy is named **_predictive replacement_** in the sense that it attempts to predict from data when a machine is close to a failure and makes you act accordingly.   

The major challenge in predictive replacement consists in establishing the relevant metrics to observe. 
In fact, we rarely have at our disposal a metric that directly captures the remaining life of a certain machine. 
The latter must be inferred through the observable metrics at our disposal, by means of an application-based extrapolation. 
Hence, the role of the data scientist should first of all understand which observable metrics are correlated to the time-to-failure 
of a machine and then establish under which circumstances it is appropriate to raise a warning and suggest a deeper checking.


## Motivation and case study

In this project we aim to understand whether predictive replacement can be used in forecasting failures of hard drive disks (HDDs). It is expected that by 2020 70% of all data and 90% of all data center data will be stored in HDDs [^a](https://www.forbes.com/sites/tomcoughlin/2017/10/11/mamr-hard-disk-drives-enable-future-data-centers/#6db918322054), 
thus preventing the failure of HDDs will play a key role in designing efficient data centers. 

Unfortunately, the time-to-failure of an HDD cannot be easily determined. By placing sensors on an HDD we may capture 
the Self-Monitoring, Analysis, and Reporting Technology (S.M.A.R.T.) metrics indicating whether the performance of the HDD are up to the required standards. We aim to understand whether and which of the S.M.A.R.T. metrics are correlated to the time-to-failure and eventually be able to raise warnings if the behavior of the HDD is suspicious. 
Obviously, we do not want to raise too many false alarms, but false alarms are definitely less costly with respect to a missed failure and a trade-off has to be determined.

In a recent Google study [^PWB07](https://static.googleusercontent.com/media/research.google.com/en//archive/disk_failures.pdf) it as been observed that 

> Further, 36% of failed drives did so without recording any S.M.A.R.T. error at all, except the temperature, meaning  that S.M.A.R.T. data alone was of limited usefulness in anticipating failures.

This means that predictive replacement alone is not enough to capture every failures, nevertheless the silver lining is that the remaining 64% of the failures were somehow (perhaps minimally) correlated to a certain level of disruption in at least one of the S.M.A.R.T. metric. Thus a constant monitoring of these metrics, together with a thorough statistical analysis may drastically improve the failure prevention of HDDs and thus the reliability of data centers. 

## Dataset description

In this project we rely on the open dataset made available by [Backblaze](https://www.backblaze.com/b2/hard-drive-test-data.html), a data storage provider which allows the user to back up data continuously, manually, when the computer is idle, or on an hourly schedule. We focused on the data relative to the period 2015-2017. 
The dataset for these three-years period consists of 7.2 millions of entries in which the HDDs are monitored on a daily basis. Each entry details the following features:

- Serial Number: The unique ID of the HDD observed.
- Model: The model of the HDD observed.
- Capacity: The capacity of the HDD observed (same models have same capacity).
- Date: The time-stamp of the observation (measures are sampled on a daily basis).
- Failure: A binary value, if equal to $0$ the observed HDD did not fail on such day, if equal to $1$ it did.
- SMART $n$ raw: Raw value of the SMART metric $n$ of the HDD observed.
- SMART $n$ normalized: Normalized value of the SMART metric $n$ of the HDD observed.

The SMART metrics reported are 45 and are model dependent. Observe that the same SMART metric for two different models may have different meaning and thus be measured differently. A specific model may lack a specific SMART metric which is indicated by NaN in the dataset.
The raw values for the SMART metrics are real numbers whose meaning depends on the specific metric and model type. The normalized counterpart takes an integer
value which ranges from $1$ to $253$ (with $1$ representing the worst case, $253$ representing the best, and $100$ representing the typical behavior). 

Observe that with respect to the study [^PWB07](https://static.googleusercontent.com/media/research.google.com/en//archive/disk_failures.pdf) the sensing is done in a much sparser way, here we have daily measurements there they had measures every few minutes. Thus we do not expect to capture as many failures as they did, nevertheless we aim to provide a computationally, memory, and energy-efficient
procedure to support the nevertheless essential preventive replacement approach.

## Structure of the report

The project is structured as follows:

- In [Notebook 2](https://github.com/FabCec/planned_replacements/blob/master/2.%20Data%20wrangling.ipynb) we wrangle the raw dataset in order to obtain manageable dataframes suitable for the analysis. In particular, given a specific model, we determine which SMART metrics are meaningful and discard the non-informative ones.

- In [Notebook 3](https://github.com/FabCec/planned_replacements/blob/master/3.%20Exploratory%20data%20analysis%20and%20inferential%20statistic.ipynb) we perform a preliminary exploratory data analysis to determine the magnitude of the correlations between the SMART metric evolution and the time-to-failure. We further investigate these correlation through solid inferential statistic methodologies

- In [Notebook 4](https://github.com/FabCec/planned_replacements/blob/master/4.%20Machine%20learning%20and%20optimization%20problem.ipynb) we apply machine learning techniques to determine policies establishing whether HDDs are close to failure and should therefore be turned off. Three different policies will be proposed:
  * Classifier based on daily observations.
  * Classifier based on time-series (multiple days) observations. 
  * Classifier based on survival regression. 





# Conclusion

The analysis performed showed that a predictive replacement policy based on such a sparse observation of the SMART metrics may provide a significant increase in the revenue only when the cost per failure is huge with respect to the daily gain of a functioning HDD, i.e., when the loss of revenue due to a wrong termination of a healthy HDD is much lower than the cost per failure.   

The problem we are facing may be cast as an anomaly detection problem, in which the anomalies are related to the values of the SMART metrics associated to HDDs close to failure. However, in addition to the class imbalance tyical to these kind of problems, we also had to deal with the noiseness due to the vast majority of HDDs failing without showing any uncommon symptom. In our opinion this is due to the sparsity of the data, the values of the SMART metrics are expected to drop as the failure approaches, but we believe this decrease needs to be captured through a finer distribution of the observations. 

The techniques presented can be promptly adapted to datasets with less sparse observations and, even in this case, provide an improvement of 3%, 6%, and 11% as the ratio between cost per failure and daily gain per active HDD is equal to 5000, 7500, and 10000, respectively. 